In [0]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
data = pd.read_csv('drive/My Drive/DL_ICP5/Sentiment.csv')
data = data[['text','sentiment']]

In [0]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

In [0]:
def createmodel():
    model = Sequential()
    lstm_out=196
    embed_dim=128
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
    #print(model.summary())

In [10]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
print(integer_encoded)
print(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [11]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print('score',score)
print('accuracy',acc)
print('Model Metrics Names',model.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
 - 18s - loss: 0.8282 - accuracy: 0.6470
Epoch 2/20
 - 16s - loss: 0.6831 - accuracy: 0.7137
Epoch 3/20
 - 16s - loss: 0.6216 - accuracy: 0.7404
Epoch 4/20
 - 16s - loss: 0.5755 - accuracy: 0.7621
Epoch 5/20
 - 16s - loss: 0.5349 - accuracy: 0.7791
Epoch 6/20
 - 16s - loss: 0.4979 - accuracy: 0.7958
Epoch 7/20
 - 15s - loss: 0.4640 - accuracy: 0.8091
Epoch 8/20
 - 16s - loss: 0.4294 - accuracy: 0.8240
Epoch 9/20
 - 16s - loss: 0.4089 - accuracy: 0.8342
Epoch 10/20
 - 16s - loss: 0.3756 - accuracy: 0.8511
Epoch 11/20
 - 16s - loss: 0.3552 - accuracy: 0.8579
Epoch 12/20
 - 15s - loss: 0.3405 - accuracy: 0.8623
Epoch 13/20
 - 16s - loss: 0.3242 - accuracy: 0.8654
Epoch 14/20
 - 15s - loss: 0.3023 - accuracy: 0.8770
Epoch 15/20
 - 16s - loss: 0.2899 - accuracy: 0.8821
Epoch 16/20
 - 16s - loss: 0.2812 - accuracy: 0.8859
Epoch 17/20
 - 17s - loss: 0.2680 - accuracy: 0.8900
Epoch 18/20
 - 17s - loss: 0.2622 - accuracy: 0.8955
Epoch 19/20
 - 16s - loss: 0.2498 - accuracy: 0.8978
Ep

In [0]:
model.save('Vishnu.h5')

In [13]:
from keras.models import load_model
model= load_model('Vishnu.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [14]:
import numpy as np
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing']
#vectorizing the tweet by the pre-fitted tokenizer instance
sentence = tokenizer.texts_to_sequences(sentence)
#padding the tweet to have exactly the same shape as `embedding_2` input
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment = model.predict_classes(sentence,batch_size=1,verbose = 2)[0]
print(sentiment)
if sentiment == 1:
  print("Neutral")
elif sentiment == 0:
  print("Negative")
elif sentiment == 2:
  print("Positive")
else:
  print("Can not be determined")

2
Positive


2. Apply GridSearchCV on the source code provided in the class

In [15]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
from sklearn.model_selection import GridSearchCV
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train, y=Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 42s - loss: 0.8309 - accuracy: 0.6438
Epoch 1/1
 - 43s - loss: 0.8256 - accuracy: 0.6442
Epoch 1/1
 - 43s - loss: 0.8306 - accuracy: 0.6416
Epoch 1/1
 - 44s - loss: 0.8306 - accuracy: 0.6426
Epoch 1/1
 - 43s - loss: 0.8293 - accuracy: 0.6402
Epoch 1/2
 - 43s - loss: 0.8298 - accuracy: 0.6402
Epoch 2/2
 - 42s - loss: 0.6854 - accuracy: 0.7057
Epoch 1/2
 - 42s - loss: 0.8249 - accuracy: 0.6455
Epoch 2/2
 - 41s - loss: 0.6863 - accuracy: 0.7093
Epoch 1/2
 - 41s - loss: 0.8311 - accuracy: 0.6429
Epoch 2/2
 - 41s - loss: 0.6847 - accuracy: 0.7104
Epoch 1/2
 - 41s - loss: 0.8333 - accuracy: 0.6402
Epoch 2/2
 - 41s - loss: 0.6823 - accuracy: 0.7103
Epoch 1/2
 - 42s - loss: 0.8222 - accuracy: 0.6448
Epoch 2/2
 - 41s - loss: 0.6763 - accuracy: 0.7174
Epoch 1/1
 - 20s - loss: 0.8328 - accuracy: 0.6422
Epoch 1/1
 - 20s - loss: 0.8368 - accuracy: 0.6419
Epoch 1/1
 - 21s - loss: 0.8346 - accuracy: 0.6408
Epoch 1/1
 - 20s - loss: 0.8337 - accuracy: 0.6459
Epoch 1/1
 - 20s - loss: 0.8304

3. Apply the code on spamdata set availablein thesourcecode (text classification on the spam.csvdata set)

In [0]:
data1 = pd.read_csv('drive/My Drive/DL_ICP5//spam.csv',encoding="ISO-8859-1")

# Keeping only the neccessary columns
data1 = data1[['v1','v2']]

In [0]:
data1['v2'] = data1['v2'].apply(lambda x: x.lower())
data1['v2'] = data1['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data1.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data1['v2'].values)
X = tokenizer.texts_to_sequences(data1['v2'].values)

X = pad_sequences(X)

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data1['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [0]:
def createmodel1():
  embed_dim=128
  lstm_out=196
  model = Sequential()
  model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(2,activation='sigmoid'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model

In [24]:
batch_size = 32
model1 = createmodel1()
model1.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)
score,acc = model1.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model1.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 32s - loss: 0.1894 - accuracy: 0.9376
Epoch 2/5
 - 31s - loss: 0.0453 - accuracy: 0.9861
Epoch 3/5
 - 31s - loss: 0.0233 - accuracy: 0.9933
Epoch 4/5
 - 31s - loss: 0.0113 - accuracy: 0.9973
Epoch 5/5
 - 31s - loss: 0.0130 - accuracy: 0.9973
0.08230023751721037
0.9738988876342773
['loss', 'accuracy']
